<a href="https://colab.research.google.com/github/knkarthik01/nystagmus-photosensitivity-ai/blob/main/notebooks/2_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📓 2_Model_Training.ipynb

Train a Dual-Branch CNN model on synthetic photosensitivity risk dataset.

🚀 Working with Data (CSV Handling) - Flexible Approaches
After running 1_data_preprocessing.ipynb, you have two options to make preprocessed_data.csv available for the next steps:

## Option 1: Manual Upload (Recommended)
Download preprocessed_data.csv from Colab:

Click on the folder 📁 icon (sidebar)

Right-click preprocessed_data.csv → Download

Upload to GitHub:

Go to your repo → Navigate to data/preprocessed/

Click Add file → Upload files

Commit the uploaded file.

In 2_model_training.ipynb, load it like:

python
Copy
Edit
data = pd.read_csv('data/preprocessed/preprocessed_data.csv')

### Option 2: Temporary Usage (within same Colab session)
If you plan to run all notebooks one after another in the same Colab session, the file stays available.

Simply run 1_data_preprocessing.ipynb first, then 2_model_training.ipynb.

### Load it directly:

python
Copy
Edit
data = pd.read_csv('preprocessed_data.csv')

⚡ Note: If you disconnect the Colab runtime, the file is lost.



In [1]:
# Step 1: Import libraries
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

In [3]:
# Step 2: Load preprocessed data
data = pd.read_csv('preprocessed_data.csv')
X = data[['brightness_level', 'eye_movement_variance']].values
y = data['photosensitivity_risk'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Step 3: Prepare PyTorch datasets
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [6]:
# Step 4: Define the Dual-Branch CNN model (simple version)
class DualBranchNet(nn.Module):
    def __init__(self):
        super(DualBranchNet, self).__init__()
        self.fc1 = nn.Linear(2, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

model = DualBranchNet()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
# Step 5: Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

Epoch 1, Loss: 3.1980
Epoch 2, Loss: 1.1958
Epoch 3, Loss: 0.8467
Epoch 4, Loss: 0.7264
Epoch 5, Loss: 0.7085
Epoch 6, Loss: 0.7176
Epoch 7, Loss: 0.7111
Epoch 8, Loss: 0.7172
Epoch 9, Loss: 0.7055
Epoch 10, Loss: 0.7086


In [8]:
# Step 6: Save the model
torch.save(model.state_dict(), 'dual_branch_cnn.pth')
print('✅ Model trained and saved as dual_branch_cnn.pth')

✅ Model trained and saved as dual_branch_cnn.pth
